In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.signal import welch

from spectools import lpsd
from spectools.ltf import LTFObject

import multiprocessing as mp
pool = mp.Pool()

In [ ]:
N = int(1e6)
fs = 2
olap = 0.75
bmin = 5.3
Lmin = 1000
Jdes = 1000
Kdes = 100
band = None

In [ ]:
fmin = fs/N
fmax = fs/2
f_linspace = np.linspace(fmin, fmax, Jdes)
f_logspace = np.logspace(np.log10(fmin), np.log10(fmax), Jdes)
i = np.arange(1,Jdes+1)
i_n = i/len(i)

target_nf = Jdes
min_Jdes = 100
max_Jdes = 1000000

In [ ]:
lpsd_obj = LTFObject(N=N, fs=fs, olap=olap, bmin=bmin, Lmin=Lmin, Jdes=Jdes, Kdes=Kdes, scheduler="lpsd")
ltf_obj = LTFObject(N=N, fs=fs, olap=olap, bmin=bmin, Lmin=Lmin, Jdes=Jdes, Kdes=Kdes, scheduler="ltf")
new_obj = LTFObject(N=N, fs=fs, olap=olap, bmin=bmin, Lmin=Lmin, Jdes=Jdes, Kdes=Kdes, scheduler="new_ltf")

lpsd_obj.adjust_Jdes_to_target_nf(target_nf, min_Jdes, max_Jdes)
ltf_obj.adjust_Jdes_to_target_nf(target_nf, min_Jdes, max_Jdes)
new_obj.adjust_Jdes_to_target_nf(target_nf, min_Jdes, max_Jdes)

fo, ro, bo, Lo, Ko, Do, Oo, nfo = lpsd_obj.get_plan()
f0, r0, b0, L0, K0, D0, O0, nf0 = ltf_obj.get_plan()
f1, r1, b1, L1, K1, D1, O1, nf1 = new_obj.get_plan()

f2i = interp1d(f_logspace, i_n, fill_value=(np.nan, np.nan), bounds_error=False)

io = np.arange(nfo)
i0 = np.arange(nf0)
i1 = np.arange(nf1)
il_n = f2i(f_linspace)
io_n = f2i(fo)
i0_n = f2i(f0)
i1_n = f2i(f1)
dfo = np.diff(fo)
df0 = np.diff(f0)
df1 = np.diff(f1)

names = ['r', 'b', 'L', 'K', 'O', 'df']
lpsd_output = (ro, bo, Lo, Ko, Oo, dfo)
ltf_output = (r0, b0, L0, K0, O0, df0)
new_output = (r1, b1, L1, K1, O1, df1)

# Interpolate scheduler outputs to common frequency grid:
res = {} 
for name, itemo, item0, item1 in zip(names, lpsd_output, ltf_output, new_output):
    print(f"Interpolating {name}")
    if name == 'df':
        interpo = interp1d(fo[:-1], dfo, kind='linear', fill_value=(np.nan, np.nan), bounds_error=False)
        interp0 = interp1d(f0[:-1], df0, kind='linear', fill_value=(np.nan, np.nan), bounds_error=False)
        interp1 = interp1d(f1[:-1], df1, kind='linear', fill_value=(np.nan, np.nan), bounds_error=False)
    else:
        interpo = interp1d(fo, itemo, kind='linear', fill_value=(np.nan, np.nan), bounds_error=False)
        interp0 = interp1d(f0, item0, kind='linear', fill_value=(np.nan, np.nan), bounds_error=False)
        interp1 = interp1d(f1, item1, kind='linear', fill_value=(np.nan, np.nan), bounds_error=False)
    res[name+'o'] = interpo(f_logspace)
    res[name+'0'] = interp0(f_logspace)
    res[name+'1'] = interp1(f_logspace)

r_DFTo = fs/res['Lo']
r_DFT0 = fs/res['L0']
r_DFT1 = fs/res['L1']

In [ ]:
print(f"Desired frequencies: {Jdes}; LPSD scheduler: {nfo}; LTF scheduler: {nf0}; New scheduler: {nf1}")

In [ ]:
print(f"Maximum frequency: {fs/2}; LPSD scheduler: {fo[-1]:6.4f}; LTF scheduler: {f0[-1]:6.4f}; New scheduler: {f1[-1]:6.4f}")

In [ ]:
print(f"Minimum length: {Lmin}; LPSD scheduler: {np.min(Lo)}; LTF scheduler: {np.min(L0)}; New scheduler: {np.min(L1)}")

In [ ]:
figsize=(6,4)
dpi=150
fontsize=8
linewidth=2.5
xlabel="Frequency index, j"
ylabel="Frequency, f(j) (Hz)"

fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
ax.axhline(y=fmin, c='lightgray', ls='-', lw=6)
ax.axhline(y=fmax, c='lightgray', ls='-', lw=6)
ax.plot(i, f_linspace, label=f"Linspace(fmin, fmax, {Jdes})", c="black", ls='--')
ax.plot(i, f_logspace,label=f"Logspace(fmin, fmax, {Jdes})", c="gray", ls='--')
ax.plot(io, fo, lw=linewidth, label=f"LPSD scheduler, Jdes={lpsd_obj.Jdes}", c="deepskyblue", ls='-')
ax.plot(i0, f0, lw=linewidth, label=f"LTF scheduler, Jdes={ltf_obj.Jdes}", c="lime", ls='--')
ax.plot(i1, f1, lw=linewidth, label=f"New scheduler, Jdes={new_obj.Jdes}", c="tomato", ls='-.')
ax.set_xlabel(xlabel, fontsize=fontsize)
ax.set_ylabel(ylabel, fontsize=fontsize)
ax.tick_params(labelsize=fontsize)
ax.grid()
ax.legend(loc='best', edgecolor='black', fancybox=True, shadow=True, framealpha=1, fontsize=fontsize, handlelength=3.)
# ax.set_xscale('log')
ax.set_yscale('log')
title=f"Kdes={Kdes}; Lmin={Lmin}; bmin={bmin}; olap={olap}; N={N}; fs={fs} Hz"
ax.set_title(title, fontsize=fontsize)
fig.tight_layout()
fig.align_ylabels()
plt.show()

In [ ]:
figsize=(6,4)
dpi=150
fontsize=8
linewidth=2.0
xlabel="Frequency index"
ylabel="Resolution bandwidth (Hz)"

fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
ax.axhline(y=fs/N, color='lightblue', ls='-', lw=6, label='Minimum resolution')
ax.axhline(y=fs/N*(1+(1-olap)*(Kdes-1)), color='pink', ls='-', lw=6, label='r_avg from K_avg')
ax.plot(i_n, res['ro'], lw=linewidth, label="LPSD scheduler", c="deepskyblue", ls='-')
ax.plot(i_n, res['r0'], lw=linewidth, label="LTF scheduler", c="lime", ls='--')
ax.plot(i_n, res['r1'], lw=linewidth, label="New scheduler", c="tomato", ls='-.')
ax.set_xlabel(xlabel, fontsize=fontsize)
ax.set_ylabel(ylabel, fontsize=fontsize)
ax.tick_params(labelsize=fontsize)
ax.grid()
ax.legend(loc='best', edgecolor='black', fancybox=True, shadow=True, framealpha=1, fontsize=fontsize, handlelength=2.5)
# ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(0,1.1)
title=f"Jdes={Jdes}; Kdes={Kdes}; Lmin={Lmin}; bmin={bmin}; olap={olap}; N={N}; fs={fs}"
ax.set_title(title, fontsize=fontsize)
fig.tight_layout()
fig.align_ylabels()
plt.show()

In [ ]:
figsize=(6,4)
dpi=150
fontsize=8
linewidth=2.5
xlabel="Frequency index, j"
ylabel="Resolution bandwidth, r(j) (Hz)"

fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
ax.axhline(y=fs/N, color='lightgray', ls='-', lw=6)
ax.plot(io, ro, lw=linewidth, label="LPSD scheduler", c="deepskyblue", ls='-')
ax.plot(i0, r0, lw=linewidth, label="LTF scheduler", c="lime", ls='--')
ax.plot(i1, r1, lw=linewidth, label="New scheduler", c="tomato", ls='-.')
ax.set_xlabel(xlabel, fontsize=fontsize)
ax.set_ylabel(ylabel, fontsize=fontsize)
ax.tick_params(labelsize=fontsize)
ax.grid()
ax.legend(loc='best', edgecolor='black', fancybox=True, shadow=True, framealpha=1, fontsize=fontsize, handlelength=3.)
# ax.set_xscale('log')
ax.set_yscale('log')
title=f"Jdes={Jdes}; Kdes={Kdes}; Lmin={Lmin}; bmin={bmin}; olap={olap}; N={N}; fs={fs} Hz"
ax.set_title(title, fontsize=fontsize)
fig.tight_layout()
fig.align_ylabels()
plt.show()

In [ ]:
figsize=(6,4)
dpi=150
fontsize=8
linewidth=2.0
xlabel="Frequency index"
ylabel=r"Relative errors in constraints ($\times 10^{14}$)"

r_DFT0 = fs/L0
r_DFT1 = fs/L1

RE = np.abs(r1[:-1]-df1)/df1
REp = np.abs(r1[1:]-df1)/df1
re = np.minimum(RE,REp)

fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
ax.plot(i1[:-1], re*1e14, lw=linewidth, c="black", label="r[j] = f[j+1] - f[j], new scheduler")
ax.plot(i1, np.abs(r1-(fs/L1))/(fs/L1)*1e14, lw=linewidth, c="tomato", ls='-', label="r[j] = fs/L[j], new scheduler")
ax.plot(i0[:-1], np.abs(r0[:-1]-df0)/df0*1e14, lw=linewidth, c="cyan", ls='--', label='r[j] = f[j+1] - f[j], LTF scheduler')
ax.plot(i0, np.abs(r0-(fs/L0))/(fs/L0)*1e14, lw=linewidth, c="b", ls='--', label="r[j] = fs/L[j], LTF scheduler")
ax.set_xlabel(xlabel, fontsize=fontsize)
ax.set_ylabel(ylabel, fontsize=fontsize)
ax.tick_params(labelsize=fontsize)
ax.grid()
ax.legend(loc='best', edgecolor='black', fancybox=True, shadow=True, framealpha=1, fontsize=fontsize, handlelength=2.5)
# ax.set_xscale('log')
# ax.set_yscale('log')
title=f"Jdes={Jdes}; Kdes={Kdes}; Lmin={Lmin}; bmin={bmin}; olap={olap}; N={N}; fs={fs} Hz"
ax.set_title(title, fontsize=fontsize)
fig.tight_layout()
fig.align_ylabels()
plt.show()

In [ ]:
figsize=(6,4)
dpi=150
fontsize=8
linewidth=1.5
xlabel="Frequency index"
ylabel="Non-integer frequency bin"

fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
ax.axhline(y=bmin, color='pink', linestyle='-', lw=6, label='Minimum bin')
ax.axhline(y=fs/2*b1[-1], color='lightgreen', ls='-', lw=6, label='Maximum bin')
ax.plot(i_n, res['b1'], lw=linewidth, label="New scheduler", color="black")
ax.plot(i_n, res['b0'], lw=linewidth, label="LTF scheduler", color="gray", ls='--')
ax.plot(i_n, res['bo'], lw=linewidth, label="LPSD scheduler", color="pink", ls='--')
ax.set_xlabel(xlabel, fontsize=fontsize)
ax.set_ylabel(ylabel, fontsize=fontsize)
ax.tick_params(labelsize=fontsize)
ax.grid()
ax.legend(loc='best', edgecolor='black', fancybox=True, shadow=True, framealpha=1, fontsize=fontsize, handlelength=2.5)
# ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(0,1.1)
fig.tight_layout()
fig.align_ylabels()
plt.show()

In [ ]:
figsize=(6,4)
dpi=150
fontsize=8
linewidth=1.5
xlabel="Frequency index"
ylabel="Non-integer frequency bin"

fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
# ax.axvline(x=fc, lw=3, ls='-', c='lightgray')
ax.axhline(y=bmin, color='pink', linestyle='-', lw=6, label='Minimum bin')
ax.axhline(y=fs/2*b1[-1], color='lightgreen', ls='-', lw=6, label='Maximum bin')
ax.plot(i1, b1, lw=linewidth, label="New scheduler", color="black")
ax.plot(i0, b0, lw=linewidth, label="LTF scheduler", color="gray", ls='--')
ax.plot(io, bo, lw=linewidth, label="LPSD scheduler", color="pink", ls='--')
ax.set_xlabel(xlabel, fontsize=fontsize)
ax.set_ylabel(ylabel, fontsize=fontsize)
ax.tick_params(labelsize=fontsize)
ax.grid()
ax.legend(loc='best', edgecolor='black', fancybox=True, shadow=True, framealpha=1, fontsize=fontsize, handlelength=2.5)
# ax.set_xscale('log')
ax.set_yscale('log')
# ax.set_xlim(0,1.1)
fig.tight_layout()
fig.align_ylabels()
plt.show()

In [ ]:
figsize=(6,4)
dpi=150
fontsize=8
linewidth=1.5
xlabel="Frequency index"
ylabel="Segment length"

fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
ax.axhline(y=Lmin, lw=6, c='lightblue', ls='-', label='Minimum length')
ax.plot(i_n, res['L1'], lw=linewidth, label="Segment length, new scheduler", c="black", ls='-')
ax.plot(i_n, res['K1'], lw=linewidth, label="Number of averages, new scheduler", c="blue", ls='-')
ax.plot(i_n, res['L0'], lw=linewidth, label="Segment length, LTF scheduler", c="gray", ls='--')
ax.plot(i_n, res['K0'], lw=linewidth, label="Number of averages, LTF scheduler", c="lightblue", ls='--')
ax.plot(i_n, res['Lo'], lw=linewidth, label="Segment length, LPSD scheduler", c="pink", ls='--')
ax.plot(i_n, res['Ko'], lw=linewidth, label="Number of averages, LPSD scheduler", c="deepskyblue", ls='--')
ax.set_xlabel(xlabel, fontsize=fontsize)
ax.set_ylabel(ylabel, fontsize=fontsize)
ax.tick_params(labelsize=fontsize)
ax.grid()
ax.legend(loc='best', edgecolor='black', fancybox=True, shadow=True, framealpha=1, fontsize=fontsize-2, handlelength=2.5)
# ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(0,1.1)
fig.tight_layout()
fig.align_ylabels()
plt.show()

In [ ]:
figsize=(6,4)
dpi=150
fontsize=8
linewidth=1.5
xlabel="Frequency index"
ylabel="Segment length"

fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
ax.axhline(y=Lmin, lw=6, c='lightblue', ls='-', label='Minimum length')
ax.plot(i1, L1, lw=linewidth, label="Segment length, new scheduler", c="black", ls='-')
ax.plot(i1, K1, lw=linewidth, label="Number of averages, new scheduler", c="blue", ls='-')
ax.plot(i0, L0, lw=linewidth, label="Segment length, LTF scheduler", c="gray", ls='--')
ax.plot(i0, K0, lw=linewidth, label="Number of averages, LTF scheduler", c="lightblue", ls='--')
ax.plot(io, Lo, lw=linewidth, label="Segment length, LPSD scheduler", c="pink", ls='--')
ax.plot(io, Ko, lw=linewidth, label="Number of averages, LPSD scheduler", c="deepskyblue", ls='--')
ax.set_xlabel(xlabel, fontsize=fontsize)
ax.set_ylabel(ylabel, fontsize=fontsize)
ax.tick_params(labelsize=fontsize)
ax.grid()
ax.legend(loc='upper right', edgecolor='black', fancybox=True, shadow=True, framealpha=1, fontsize=fontsize-2, handlelength=2.5)
# ax.set_xscale('log')
ax.set_yscale('log')
fig.tight_layout()
fig.align_ylabels()
plt.show()

In [ ]:
figsize=(6,4)
dpi=150
fontsize=8
linewidth=1.5
xlabel="Frequency index"
ylabel="Fractional overlap"
title=""

fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
ax.axhline(y=olap, c='lightblue', linestyle='-', lw=6, label='Desired overlap')
ax.plot(i_n, res['O1'], lw=linewidth, label="Actual overlap, new scheduler", c="black")
ax.plot(i_n, res['O0'], lw=linewidth, label="Actual overlap, LTF scheduler", c="gray", ls='--')
ax.plot(i_n, res['Oo'], lw=linewidth, label="Actual overlap, LPSD scheduler", c="pink", ls='--')
ax.set_xlabel(xlabel, fontsize=fontsize)
ax.set_ylabel(ylabel, fontsize=fontsize)
ax.set_title(title, fontsize=fontsize)
ax.tick_params(labelsize=fontsize)
ax.grid()
ax.legend(loc='best', edgecolor='black', fancybox=True, shadow=True, framealpha=1, fontsize=fontsize, handlelength=2.5)
# ax.set_xscale('log')
# ax.set_yscale('log')
fig.tight_layout()
fig.align_ylabels()
plt.show()

In [ ]:
figsize=(6,4)
dpi=150
fontsize=8
linewidth=1.5
xlabel="Frequency index"
ylabel="Fractional overlap"
title=""

fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
ax.axhline(y=olap, c='lightblue', linestyle='-', lw=6, label='Desired overlap')
ax.plot(i1, O1, lw=linewidth, label="Actual overlap, new scheduler", c="black")
ax.plot(i0, O0, lw=linewidth, label="Actual overlap, LTF scheduler", c="gray", ls='--')
ax.plot(io, Oo, lw=linewidth, label="Actual overlap, LPSD scheduler", c="pink", ls='--')
ax.set_xlabel(xlabel, fontsize=fontsize)
ax.set_ylabel(ylabel, fontsize=fontsize)
ax.set_title(title, fontsize=fontsize)
ax.tick_params(labelsize=fontsize)
ax.grid()
ax.legend(loc='best', edgecolor='black', fancybox=True, shadow=True, framealpha=1, fontsize=fontsize, handlelength=2.5)
# ax.set_xscale('log')
# ax.set_yscale('log')
fig.tight_layout()
fig.align_ylabels()
plt.show()

In [ ]:
def psd(f):
    psd_values = np.zeros_like(f)
    psd_values += 1e-12 / (f + 1e-6)
    psd_values += 1e-11 * (f - 1e-3)
    low_freq_peak = 1.23e-4
    psd_values += 1e-3 * np.exp(-((f - low_freq_peak) / (low_freq_peak / 5))**2)
    high_freq_peak = 2.5e-2
    psd_values += 1e-3 * np.exp(-((f - high_freq_peak) / (high_freq_peak / 50))**2)
    notch_center = 0.15
    notch_width = notch_center / 10
    notch_depth = 0.8
    notch = np.exp(-((f - notch_center) / notch_width)**2)
    psd_values *= 1 - notch_depth * notch
    psd_values *= (1 + np.random.rand(len(psd_values)))
    return psd_values

freqs = np.logspace(-7,1,1000)

plt.figure(figsize=(10, 6))
plt.loglog(freqs, np.sqrt(psd(freqs)))
plt.xlabel('Frequency (Hz)')
plt.ylabel('Voltage spectral density (Vrms/√Hz)')
plt.title('Generated Signal Spectral Density')
plt.grid(True)
plt.show()

In [ ]:
frequencies = np.fft.fftfreq(N, 1/fs)
psd_values = psd(np.abs(frequencies))
random_phases = np.exp(1j * 2 * np.pi * np.random.rand(len(frequencies)))
frequency_signal = np.sqrt(psd_values) * random_phases
time_series = np.fft.ifft(frequency_signal).real

figsize=(8,3)
dpi=150
fontsize=8
linewidth=1.
xlabel="Sample"
ylabel="Signal"

fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
ax.plot(time_series, linewidth=linewidth, label="test", color="tomato")
ax.set_xlabel(xlabel, fontsize=fontsize)
ax.set_ylabel(ylabel, fontsize=fontsize)
ax.tick_params(labelsize=fontsize)
ax.grid()
fig.tight_layout()
fig.align_ylabels()
plt.show()

In [ ]:
f, Pxx = welch(time_series, fs, nperseg=int(N/2), noverlap=int(N/4), scaling='density', return_onesided=True)
f, PS = welch(time_series, fs, nperseg=int(N/2), noverlap=int(N/4), scaling='spectrum', return_onesided=True)

pso = lpsd.ltf(time_series, fs=fs, bmin=bmin, Jdes=Jdes, Kdes=Kdes, Lmin=Lmin, pool=pool, win=np.kaiser, olap=olap, scheduler='lpsd', adjust_Jdes=True)
ps0 = lpsd.ltf(time_series, fs=fs, bmin=bmin, Jdes=Jdes, Kdes=Kdes, Lmin=Lmin, pool=pool, win=np.kaiser, olap=olap, scheduler='ltf', adjust_Jdes=True)
ps1 = lpsd.ltf(time_series, fs=fs, bmin=bmin, Jdes=Jdes, Kdes=Kdes, Lmin=Lmin, pool=pool, win=np.kaiser, olap=olap, scheduler='new_ltf', adjust_Jdes=True)

In [ ]:
print(f"Desired frequencies: {Jdes}; LPSD scheduler: {pso.nf}; LTF scheduler: {ps0.nf}; New scheduler: {ps1.nf}")

In [ ]:
lw = 1
fontsize = 8

fig, ax = plt.subplots(figsize=(6,4), dpi=150)
ax.plot(f, np.sqrt(Pxx), c='g', lw=1, ls='-', label='Welch')
ax.plot(pso.f, pso.asd, c='k', lw=3, ls='-', label='LPSD scheduler')
ax.plot(ps0.f, ps0.asd, c='tomato', lw=2, ls='-', label='LTF scheduler')
ax.plot(ps1.f, ps1.asd, c='lime', lw=1, ls='-', label='New scheduler')
ax.set_xlabel('Frequency (Hz)', fontsize=fontsize)
ax.set_ylabel(r'$\gamma_{xy}^2$', fontsize=fontsize)
# ax.set_ylim(1e-3,2)
ax.set_yscale('log')
ax.set_xscale('log')
ax.legend(loc='best', edgecolor='black', fancybox=True, shadow=True, framealpha=1, fontsize=fontsize, handlelength=3.)
title=f"Jdes={Jdes}; Kdes={Kdes}; Lmin={Lmin}; bmin={bmin}; olap={olap}; N={N}; fs={fs} Hz"
ax.set_title(title, fontsize=fontsize)
fig.tight_layout()
fig.align_ylabels()
plt.show()